In [1]:
import sys
sys.path.append('..')
from CompoundPoisson2 import CompoundPoisson2
from MCABS import MCLossAbs
import numpy as np
import scipy.stats as st
import math
from scipy.optimize import minimize
import time

In [2]:
#class for loss function of the following form :  l(x) = 1 / (1 + alpha)(sum(exp(beta * xi)) + alpha*exp(beta *sum(xi)))- (alpha+d)/(alpha+1)
class MCLossFunction2(MCLossAbs):
    def __init__(self, X, alpha):
        self.__alpha = alpha
        super(MCLossFunction2, self).__init__(X)
        
    def shortfall_risk(self, m = None):
        m = self._check_argument(m)
        #This substract from the col i of X, m_i
        #X is a matrix of dim columns and N rows where N is the sample's length
        x_minus_m = np.subtract(self.X, m)
        #axis = 1 means we sum the elements of each row
        mean_sum = np.mean(np.sum(x_minus_m, axis = 1))
        pos_part = np.maximum(x_minus_m, 0)
        mean_sum_2 = np.mean(np.sum(pos_part ** 2, axis = 1))
        cross_term = 0.
        for i in range(self.dim):
            for j in range(i + 1, self.dim):
                cross_term += np.mean(np.multiply(pos_part[:, i], pos_part[:, j]))
        return mean_sum + 0.5 * mean_sum_2 + self.__alpha * cross_term
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        x_minus_m = np.subtract(self.X, m)
        pos_part = np.maximum(x_minus_m, 0) 
        pos_part_mean = np.mean(pos_part, axis = 0)
        cross_jac = []
        for i in range(self.dim):
            temp = 0.
            indic_i = np.sign(pos_part[:, i])
            for k in range(self.dim):
                if k != i:
                    temp += self.__alpha * np.mean(np.multiply(pos_part[:, k], indic_i))
            cross_jac.append(temp)
        return pos_part_mean + 1. + cross_jac

In [ ]:
#code for compound
#N is the number of steps
#the following code is for construction of compound poisson distribution
d = 10
N = 100000
T = 1
lam = np.random.randint(1, 3, d)
lam_exp = np.random.uniform(0.2, 1.2, d)
mu = [1 for _ in range(d)]
sigma = [1 for _ in range(d)]

R = np.random.randn(d, d)
cov = np.dot(R, np.transpose(R))

rho_gauss = np.ones((d, d))

for i in range(d):
    for j in range(i + 1, d):
        rho_gauss[i, j] = cov[i, j] / np.sqrt(cov[i, i] * cov[j, j])
        rho_gauss[j, i] = rho_gauss[i, j]
C = CompoundPoisson2(T, lam, rho_gauss, N, mu, sigma)
rho_pois = C.get_rho_pois()
#jump size with Gaussian distribution 
cov_matrix_X_gauss_jump = C.cov_final_X_gauss()
X_gauss_jump = C.compound_poisson2_gauss()

In [ ]:
alpha = 1.
maxiter = 3500
start_time = time.time()
loss = MCLossFunction2(distr, alpha)
init = np.ones(loss.dim)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})

res = minimize(loss.objective, init, 
              jac = loss.objective_jac, 
              constraints = cons, 
               method='SLSQP',
               options={'maxiter': maxiter, 'disp': True})
print(res)
print("--- MC Method: %s seconds ---" % (time.time() - start_time))